1 extract spectra using evselect

In [ ]:
for name in ['mos1S001-allevc', 'mos2S002-allevc-nechip5', 'pnS003-allevc']:
    regdir = f'{datapath}/srcs_regions'
    os.chdir(regdir)
    regs = glob(f'{datapath}/group-05r200-exsrc_phy_{name.split("-")[0]}.reg')
    print(regs)
    f = open(f'{regdir}/group-05r200_spec_{name}.sh', 'w')

    for reg in regs:
        sf = open(reg)
        lines = sf.readlines()
        lines = [l.strip() for l in lines]
        regtxt = ''.join(lines)
        sf.close()

        regname = reg.split('/')[-1]
        if 'mos' in name:
            f.write(f'\n')
            f.write(f'evselect table={name}.fits xcolumn=X ycolumn=Y imagebinning=binSize ximagebinsize=80 yimagebinsize=80 withimageset=Y imageset={regname.split(".")[0]}_img.fits withspectrumset=Y spectrumset={regname.split(".")[0]}_spec.pi energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=11999 expression="(FLAG==0) && (PATTERN<=12) && {regtxt}"  | tee {reg.split("/")[-1].split(".")[0]}_spec.log\n')
        else:
            f.write(f'\n')
            f.write(f'evselect table={name}.fits xcolumn=X ycolumn=Y imagebinning=binSize ximagebinsize=80 yimagebinsize=80 withimageset=Y imageset={regname.split(".")[0]}_img.fits withspectrumset=Y spectrumset={regname.split(".")[0]}_spec.pi energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=20479 expression="(FLAG==0) && (PATTERN<=4) && {regtxt}"  | tee {reg.split("/")[-1].split(".")[0]}_spec.log\n')
    f.close()


2 extract rmf and arf 

In [ ]:
f = open(f'gen_rmf.sh', 'w')

specfiles = glob('group*.pi')

for specfile in specfiles:
    
    if 'mos1' in specfile:
        badpix_file = f'mos1S001-fovimt.fits'
    elif 'mos2' in specfile:
        badpix_file = f'mos2S002-fovimt.fits'
    else:
        badpix_file = f'pnS003-fovimt.fits'

    f.write(f'''
rmfgen spectrumset={specfile} rmfset={specfile.split('.')[0]}.rmf detmaptype=flat extendedsource=yes badpixlocation={badpix_file}\n
arfgen spectrumset={specfile} rmfset={specfile.split('.')[0]}.rmf arfset={specfile.split('.')[0]}.arf badpixlocation={badpix_file}\n
        ''') 

f.close()